# This is how to load OPENAI_API_KEY into .py file for use

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPEN_API_KEY is not set int the environment variables")

# Hello WOrld
### Going over how to create an agent, and run an agent

In [1]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond only in all caps.",
    model="gpt-4o-mini"
)
# ==== RUnning the agent or "invoking" it so we get a response ===
result = await Runner.run(agent, "Hello! How are you?")
result.final_output

"HELLO! I'M DOING GREAT, THANK YOU! HOW ABOUT YOU?"

# Making an agent functional:

In [7]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
    model="gpt-4o-mini"
)

topic = "Boogers"

result = await Runner.run(joke_agent, topic)
result.final_output

'Why did the booger break up with the tissue?\n\nBecause it found someone more snot-compatible!'

# Second agent that takes joke output and translates it to another language

In [21]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language translator. Youre job is to translate the language to a users query.",
    model="gpt-4o-mini"
)

joke_result = await Runner.run(joke_agent, topic)
language_result = await Runner.run(language_agent, f'Translate this text into Spanish: {joke_result.final_output}')
print(f"Joke: {joke_result.final_output}")
print(f'\n\nLanguage Result: {language_result.final_output}')


Joke: Why did the booger go to the party?

Because it heard it would be a real snot-fest!


Language Result: ¿Por qué fue el moco a la fiesta?

¡Porque escuchó que sería un verdadero festival de mocos!


# Structured Output

Structured outputs are a way to format the output that the LLM gives us in a structured manner (Like a JSON object). 

This can be useful for tasks that require specific formatting or data extraction.

Create a class and give it in the parameters: BaseModel , and then output_type= 

In [4]:
from pydantic import BaseModel
from agents import Agent, Runner

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int  # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent",
    instructions="You are an agent for creating recipes. You will be given the name of a food and your job is to output that as an actual detailed recipe. The cooking time should be in minutes.",
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Italian Sausage with Spaghetti")
recipe = response.final_output
print(recipe)


title='Italian Sausage with Spaghetti' ingredients=['12 oz (340g) spaghetti', '1 lb (450g) Italian sausage, casings removed', '2 tablespoons olive oil', '1 onion, finely chopped', '3 cloves garlic, minced', '1 can (14 oz) crushed tomatoes', '1 tablespoon tomato paste', '1 teaspoon dried oregano', '1 teaspoon dried basil', 'Salt and pepper to taste', '1/4 cup grated Parmesan cheese', 'Fresh basil leaves for garnish'] cooking_time=30 servings=4


# Tool Calling

In [50]:
from agents import function_tool

@function_tool
def get_weather(city: str) -> str:
    print(f'Getting weather for {city}')
    return f"The weather in {city} is sunny"

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temperature for {city}")
    return f'70 degrees'

agent = Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature.",
    tools=[get_weather, get_temperature]
)

result = await Runner.run(agent, "what is the weather and temperature for Dallas?")
result.final_output

Getting weather for Dallas
Getting temperature for Dallas


'The weather in Dallas is sunny, with a temperature of 70 degrees.'

## Built in tools from OpenAI-Agents SDK 
one of them is WebSearchTool

In [57]:
from openai.types.responses import ResponseTextDeltaEvent
from agents import WebSearchTool

news_agent = Agent(
    name="News Reporter",
    instructions="You a re a news reporter. Your job is to find recent news articles on the internet about US politics.",
    tools=[WebSearchTool()]
)

result = Runner.run_streamed(news_agent, input="find most recent news about Trump.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Here are the latest developments involving President Donald Trump:

**Visit to Texas Flood Zone**

On July 11, 2025, President Trump visited Kerr County, Texas, the site of a catastrophic flash flood that claimed at least 120 lives on July 4. He defended the federal and state disaster response efforts, praising Texas Governor Greg Abbott and Homeland Security Secretary Kristi Noem. Critics have raised concerns about the lack of early warning systems and federal budget cuts that may have exacerbated the disaster's impact. ([reuters.com](https://www.reuters.com/sustainability/climate-energy/trump-visit-texas-flood-site-amid-questions-about-disaster-response-2025-07-11/?utm_source=openai))

**Trade Tariffs Announcement**

President Trump has reignited trade tensions by threatening new tariffs ranging from 25% to 50% on imports from nearly two dozen countries, including South Korea, Japan, Canada, Bangladesh, and Brazil, unless they agree to bilateral trade deals. These measures, set to be

# Handoffs


In [ ]:
from agents import Agent, Runner
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline: str
    tutorial: str


tutorial_generator = Agent(
    name="Tutorial Generator",
    handoff_description="Used for generating a tutorial based on an outline.",
    instructions=(
        "Given a programming topic and an outline, your job is to generate code snippets for each section of the outline."
        "Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples."
        "Where it makes sense, include comments in the code snippets to further explain the code."
    )
)

outline_builder = Agent(
    name="Outline Builder",
    instructions=(
        "Given a programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline."
        "After making the outline, hand it to the tutorial generator agent."
    ),
    handoffs=[tutorial_generator]
)

tutorial_response = await Runner.run(outline_builder, "class in Python and how to use __init__ and self in classes")

print(tutorial_response.final_output)

## Tutorial Outline: Understanding Classes in Python - Using `__init__` and `self`

### Introduction
- Brief introduction to Object-Oriented Programming (OOP)
- Importance of classes in Python

### Section 1: What is a Class?
- Definition of a class
- Basic syntax in Python with examples
- Difference between classes and objects

### Section 2: Creating a Simple Class
- Step-by-step guide to creating a basic class
- Example code with description
- Explanation of class attributes and methods

### Section 3: Understanding the `__init__` Method
- Purpose of the `__init__` method in class initialization
- Syntax and use cases
- How `__init__` differs from other methods

#### Subsection: Practical Example
- Create a class using the `__init__` method
- Example with parameters: initializing object attributes
- Discuss different scenarios and variations

### Section 4: Exploring the `self` Keyword
- What is `self`, and why do we use it?
- Role of `self` in method definitions and object state
- 